In [1]:
import pandas as pd
from pyspark.sql.functions import first
from pyspark.sql.types import *
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession


sc = SparkContext.getOrCreate();
spark = SparkSession(sc)

In [3]:
fees_df = pd.read_excel('task1.xlsx', sheet_name='fees')
edu_fees_df = pd.read_excel('task1.xlsx', sheet_name='edu fees')
imp_fees_df = pd.read_excel('task1.xlsx', sheet_name='implementation fees')

# Select columns only needed
columns = ['Column headers', 'sample data', 'Target column headers']
fees_df = fees_df[columns]
edu_fees_df = edu_fees_df[columns]
imp_fees_df = imp_fees_df[columns]

In [4]:
imp_fees_df.head()

,Column headers,sample data,Target column headers
0,ClientName,abc,ClientName
1,Plan ID,123,Plan ID
2,FeeDescription,new plan,FeeDescription
3,ProjectStartDate,march 1st 2021,ProjectStartDate
4,ProjectEndDate,dec 5th 2021,ProjectEndDate


In [5]:
excel_schema = StructType([ StructField("sample data", StringType(), True)
                           ,StructField("Target column headers", StringType(), True)
                           ,StructField("Column headers", StringType(), True)])

In [6]:
# Load Pandas DF to Spark DF
s_fees_df = spark.createDataFrame(fees_df, excel_schema)
s_edu_fees_df = spark.createDataFrame(edu_fees_df, excel_schema)
s_imp_fees_df = spark.createDataFrame(imp_fees_df, excel_schema)

In [7]:
# GroupBy Pivot, Agg 
s_fees_df = s_fees_df.groupBy().pivot("sample data").agg(first('Target column headers'))
s_edu_fees_df = s_edu_fees_df.groupBy().pivot("sample data").agg(first('Target column headers'))
s_imp_fees_df = s_imp_fees_df.groupBy().pivot("sample data").agg(first('Target column headers'))

In [61]:
# UnionByName
final_df = s_fees_df.unionByName(s_edu_fees_df, allowMissingColumns=True).unionByName(s_imp_fees_df, allowMissingColumns=True)

In [64]:
final_df.show()

+---------------------+----------+-----------+--------------+-------------+---------------+-------+--------------+----------------+------------+
|AdiitionalDescription|ClientName|Description|FeeDescription|InvoiceAmount|    InvoiceDate|Plan ID|ProjectEndDate|ProjectStartDate|QuaterBilled|
+---------------------+----------+-----------+--------------+-------------+---------------+-------+--------------+----------------+------------+
|            plan fees|       jio|  plan fees|      new plan|          250|april 11st 2021|    567| dec 15th 2021| march 21st 2021|     Q4 2021|
|                 null|       ceo|       null|      new plan|          200|           null|    345|          null|            null|     Q2 2020|
|                 null|       abc|       null|      new plan|          100| april 1st 2021|    123|  dec 5th 2021|  march 1st 2021|     Q1 2021|
+---------------------+----------+-----------+--------------+-------------+---------------+-------+--------------+----------------